In [ ]:
!nvidia-smi

In [2]:
import shutil
import os
import cv2
import csv
import numpy as np
import pandas as pd

In [3]:
import shutil
shutil.unpack_archive('drive/MyDrive/JPEGImages-trainval.zip','data')

In [4]:
import shutil
shutil.unpack_archive('drive/MyDrive/Annotations.zip','gt')

In [ ]:
pip install rarfile

In [6]:
import rarfile
rar = rarfile.RarFile('drive/MyDrive/ProvoSuperCubeNNref.rar')
rar.extractall()

In [7]:
shutil.unpack_archive('drive/MyDrive/training-20201219T093629Z-001.zip','new')

In [12]:
for r,d,f in os.walk('/content/new/training/data/labels/train'):
  for file in f:
    if '.txt' in file:
      shutil.copy(os.path.join(r,file),'/content/new/training/data/images/train')

for r,d,f in os.walk('/content/new/training/data/labels/valid'):
  for file in f:
    if '.txt' in file:
      shutil.copy(os.path.join(r,file),'/content/new/training/data/images/valid')

In [13]:
import os
import cv2
import numpy as np
import csv
import xml.etree.ElementTree as ET
import shutil
import random


available_imgs=os.listdir('data/JPEGImages-trainval')
xml_files=[]
for r,d,f in os.walk('gt'):
    for file in f:
        if '.xml' in file:
            xml_files.append('{}/{}'.format(r,file))

data=[]
for xml_file in xml_files:
    tree = ET.parse(xml_file)
    ref_name='data/JPEGImages-trainval/'+xml_file[3:-4]+'.jpg'
    if xml_file[3:-4]+'.jpg' not in available_imgs:
      continue
    root = tree.getroot()   
    list_with_all_boxes = []    
    for boxes in root.iter('object'):    
        if boxes.find("name").text not in ["airplane","vehicle","airport"]:
          continue
        ymin, xmin, ymax, xmax = None, None, None, None
    
        ymin = int(boxes.find("bndbox/ymin").text)
        xmin = int(boxes.find("bndbox/xmin").text)
        ymax = int(boxes.find("bndbox/ymax").text)
        xmax = int(boxes.find("bndbox/xmax").text) 
        class_name=boxes.find("name").text
        list_with_single_boxes = [xmin, ymin, xmax, ymax]
        data.append([ref_name,list_with_single_boxes,class_name])

for da in data:
  if da[1][1]>=da[1][3] or da[1][0]>=da[1][2]:
    data.remove(da)
  

In [15]:
names={}
for da in data:
    if da[0] not in names:
        names[da[0]]=[]
    names[da[0]].append([da[0],da[1][0],da[1][1],da[1][2],da[1][3],da[2]])
train_names=list(names.keys())[:1000]
test_names=list(names.keys())[1000:1500]

In [16]:
"""
classes={}
for da in data:
  if da[0] in train_names:
    if da[-1] not in classes:
      classes[da[-1]]=0
    classes[da[-1]]+=1
labels_to_names={}
labels_to_names2={}
for cl in classes:
  if cl not in labels_to_names:
    labels_to_names[cl]=len(labels_to_names)
    labels_to_names2[len(labels_to_names)-1]=cl
sorted=[None for i in range(20)]
for l in labels_to_names:
sorted[labels_to_names[l]]=l
"""

In [19]:
labels_to_names={"pools":1,"vehicle":0,"airplane":2,"airport":3}

In [20]:
for key in names:
        img=cv2.imread(key)
        file1 = open("{}.txt".format(key[:-4]),"w") 
        h=img.shape[0]
        w=img.shape[1]
        for item in names[key]:
            x=round(((int(item[3])+int(item[1]))/2)/w,6)
            y=round(((int(item[4])+int(item[2]))/2)/h,6)
            dw=round((int(item[3])-int(item[1]))/w,6)
            dh=round((int(item[4])-int(item[2]))/h,6)
            L=["{} {} {} {} {}\n".format(labels_to_names[item[5]], x, y, dw, dh)]
            file1.writelines(L) 
        file1.close()   

In [22]:
file1 = open("train.txt","w") 
for r,d,f in os.walk('data'):
    for file in f:
        if '.jpg' in file and 'data/JPEGImages-trainval/{}'.format(file) in train_names:
            file1.write('data/JPEGImages-trainval/{}\n'.format(file))

for r,d,f in os.walk('/content/new/training/data/images/train'):
    for file in f:
        if '.jpg' in file:
          file1.write('new/training/data/images/train/{}\n'.format(file))
file1.close()     

file1 = open("test.txt","w") 
for r,d,f in os.walk('data'):
    for file in f:
        if '.jpg' in file and 'data/JPEGImages-trainval/{}'.format(file) in test_names:
            file1.write('data/JPEGImages-trainval/{}\n'.format(file))
for r,d,f in os.walk('/content/new/training/data/images/valid'):
    for file in f:
        if '.jpg' in file:
          file1.write('new/training/data/images/valid/{}\n'.format(file))
file1.close()  

In [ ]:
!git clone 'https://github.com/ultralytics/yolov5'

In [ ]:
cd yolov5

In [ ]:
!pip install -r requirements.txt

In [ ]:
cd ..

In [ ]:
!python 'yolov5/train.py' --img-size=800 --batch 12 --epochs 20  --data 'yolov5/data/coco.yaml' --cfg 'yolov5/models/yolov5l.yaml' --weights 'runs/train/exp3/weights/best.pt' --device 0

In [34]:
!cp '/content/runs/train/exp/weights/best.pt' 'drive/MyDrive/dior-yolov5.pt'

In [42]:
shutil.make_archive('yolov5-dior','zip','/content/runs/train/exp8')

'/content/yolov5-dior.zip'

In [43]:
!cp '/content/yolov5-dior.zip' 'drive/MyDrive'

In [ ]:
#Testing

In [25]:
!cp 'drive/MyDrive/dior-yolov5.pt' '.'

In [ ]:
import tifffile
from google.colab.patches import cv2_imshow
import numpy as np
import cv2
rgbn = tifffile.imread('ProvoSuperCubeNNref.tif')[..., 1:4]
image= 255*(rgbn / rgbn.max())
image=image.astype(np.uint8)
image=image[3500:3500+900,930:930+900]
cv2.imwrite('sat.jpg',image)

In [ ]:
!python yolov5/detect.py --img-size=4000 --source sat.jpg  --weights runs/train/exp4/weights/best.pt --conf 0.5

In [14]:
def NonOverlappingCropPatches(im, patch_size=800, stride=800): #generate patches of the image
    h,w,z = im.shape
    patches = []
    for i in range(0, h - stride, stride):
        for j in range(0, w - stride, stride):
            patch = im[i:i + patch_size, j:j + patch_size,:]
            patches.append(patch)
    return patches

In [12]:
!mkdir 'patches'

In [15]:
import tifffile
import time
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
rgbn = tifffile.imread('ProvoSuperCubeNNref.tif')[..., 1:4]
image= 255*(rgbn / rgbn.max())
image=image.astype(np.uint8)

patches=NonOverlappingCropPatches(image, patch_size=800, stride=800)
for ip,patch in enumerate(patches): #create patches 
  cv2.imwrite('patches/{}.jpg'.format(ip),patch)


In [ ]:
!python yolov5/detect.py --img-size=4000 --source patches  --weights runs/train/exp4/weights/best.pt --conf 0.5

In [26]:
import os
img=np.zeros((6*800,2*800,3),dtype=np.uint8) 
data=[]
for r,d,f in os.walk('/content/runs/detect/exp6'):
  for file in f: 
    if file not in ['10.jpg','11.jpg']:
      if '.jpg' in file:
        data.append(os.path.join(r,file))
data.sort()
data.extend(['/content/runs/detect/exp6/10.jpg','/content/runs/detect/exp6/11.jpg'])

#get the predicted patches

In [25]:
#ensemble the patch images to image
for id,item in enumerate(data): 
  temp=cv2.imread(item)
  img[(id//2)*800:(1+id//2)*800,(id%2)*800:(1+id%2)*800,:]=temp
  cv2.imwrite('im5.jpg',img)
